## Python2 与Python3 字符串的区别
在 py2 中，有两种类型字符串：str 和 unicode。但严格的来说，str 并不是完全意义上的字符串，把它称作 字节码串 更合适。unicode 则作为真正意义上的 字符串. 字符串 是一种 数据结构。数据结构 = 原数据 + 数据描述（为了更容易理解），而字符串的 “数据描述” 即为何种字符集。字节码 则是没有数据描述的。py2 中，读取 str 类型时，它只代表了一串数据，至于这串数据是何种编码，系统是不知道的，需要手动指定。而 unicode 类型，则存储的相应的字符集，系统在读取时便能根据此数据描述对数据进行识别和处理。'\xc4\xe3\xba\xc3' 对计算机来说，就是 1 个字节的字节码串在一起。u'\u4f60\u597d' 对计算机来说，则是每 2 个字节作为一个单位，去映射 unicode 字符集做运算。

在Python2 中:

In [ ]:
# -*- coding: utf-8 -*-
"""
作用是注明脚本上下文的字符集
比如当我们定义一个 str 类型的数据时
它是 utf-8 字符集的字节码
"""
str = "你好" #以 utf-8 字节码去编码

import sys
# ascii
print sys.getdefaultencoding()
"""
但不要和 python 的默认字符集搞混
默认字符集会在一些隐式加解码时被运用
"""

# 设置 python 的默认字符集
reload(sys)
sys.setdefaultencoding("utf-8")

"""
下面的代码存在一次隐式的 decode
在隐式或没有指定 decode 的字符集时
会使用 默认字符集（这里已将默认字符集设为 utf-8）进行解码
如果我们没有修改默认字符集而使用 ascii 的话
就会报错了 ascii 是没办法编码中文的
"""
"你好".encode("gb2312")


"""
py2中 write 写的为字节码串
当前源码编码字符集为 utf-8
所以写入时 "你好" 是以 utf-8 编码的
"""
with open("encode.txt", "w") as f:
    f.write("你好")

"""
读取的 str 也为 utf-8 的字节码串
"""
with open("encode.txt", "r") as f:
    str = f.read()

py3 则对字符串和字节码进行了更为规范的定义：str 和 bytes。str 终于成为了真正意义上的字符串，bytes 也形象的表征了字节码串儿（java 中的字节数组）, py3 默认支持 unicode 字符集，故不需要鸡肋的使用 py2 中的 u"xxx" 去定义一个 unicode 字符串。简单来说，py2 想定义一个日常的字符串，需要手动 u"xxx"，"xxx" 定义的是当前编码设定下的字节码串儿，典型的本末倒置。在 py3 中，不再有 u"xxx" 的语法，"xxx" 定义的就是字符串（unicode编码），b"xxx" 定义的才是字节码串儿。所以在 py3 中，coding: utf-8 的注释没有任何用处了。在 py2 中它会隐式的根据指定编码对 str 做字节码的定义，但在 py3 中已经没有这种隐式的转换和定义了。

字符串在Python3内部的表示是unicode编码，因此，在做编码转换时，通常需要以unicode作为中间编码，即先将其他编码的字符串解码（decode）成unicode，再从unicode编码（encode）成另一种编码。Decode的作用是将其他编码的字符串转换成unicode编码，如str1.decode('gb2312')，表示将gb2312编码的字符串str1转换成unicode编码。Encode的作用是将unicode编码转换成其他编码的字符串，如str2.encode('gb2312')，表示将unicode编码的字符串str2转换成gb2312编码。因此，转码的时候一定要先搞明白，字符串str是什么编码，然后decode成unicode，然后再encode成其他编码

在Python3 中:

In [ ]:
import sys

# 默认字符集由 ascii 改为 utf-8
print("默认字符集为：%s" % (sys.getdefaultencoding()))

str = "你好"

# 2 个字符长度 py2 中就不会把 str 看为数据结构 而是字节码 所以结果会是 6
print(len(str))


# 不指定字符集将使用默认的 utf-8 进行编码
str.encode() # b'\xe4\xbd\xa0\xe5\xa5\xbd'
str.encode("utf-8") # b'\xe4\xbd\xa0\xe5\xa5\xbd'
str.encode("gb2312") # b'\xc4\xe3\xba\xc3'

hello_utf8   = b'\xe4\xbd\xa0\xe5\xa5\xbd'
hello_gb2312 = b'\xc4\xe3\xba\xc3'

# utf-8 转 gb2312 / gb2312 转 utf-8
# 都是以 unicode 作为基准进行操作
hello_utf8.decode("utf-8").encode("gb2312")
hello_gb2312.decode("gb2312").encode("utf-8")

In [ ]:
py 以 unicode 作为字符运算的基准。

bytes  --  decode --  unicode --  encode --  bytes

unicode 编码 至其他字符集（utf-8/gb2312），其他字符集 解码 至 unicode

在 py2 中经常遇到的错误：

"你好".encode("utf-8")

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc4 in position 0: ordinal not in range(128)

错误的原因是什么呢？py2 以 ascii 作为默认字符集，"你好" 直接 encode 至其他编码时会进行一次隐式的解码。因为 py 以 unicode 作为运算基准，所以会先 decode 至 unicode，你没使用 py 就隐式执行，使用默认字符集 ascii 做解码，自然无法解码中文字符的字节码。

而在 py3 中是不允许这种操作的。从 bytes 到另一 bytes 你必须先 decode 后才能 encode。

代码中字符串的默认编码与代码文件本身的编码一致。python2 在安装时，默认的编码是ascii，当程序中出现非ascii编码时，python的处理常常会报这样的错UnicodeDecodeError: 'ascii' codec can't decode byte 0x?? in position 1: ordinal not in range(128)，python没办法处理非ascii编码的，此时需要自己设置将python的默认编码，一般设置为utf8的编码格式。

解决方法有三中：

1.在命令行修改，仅本会话有效：
通过>>>sys.getdefaultencoding()查看当前编码(若报错，先执行>>>import sys >>>reload(sys));
通过>>>sys.setdefaultencoding('utf8')设置编码

2.较繁琐，最有效
在程序文件中以下三句
import sys
reload(sys)
sys.setdefaultencoding('utf8')

3.修改Python本环境（推荐）
在Python的Lib\site-packages文件夹下新建一个sitecustomize.py文件，内容为：
#coding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

重启Python解释器，发现编码已被设置为utf8，与方案二同效；这是因为系统在Python启动的时候，自行调用该文件，设置系统的默认编码，而不需要每次都手动加上解决代码，属于一劳永逸的解决方法。

In [10]:
class Fraction(object):
    def __init__(self, top, bottom):
        self.num = top 
        self.den = bottom
        
    def __str__(self):
        return str(self.num) + "/" + str(self.den)

    def show(self):
        print(self.num, "/", self.den)
        
    def __add__(self, otherfraction):
        newnum = self.num * otherfraction.den + \
        self.den * otherfraction.num 
        newden = self.den * otherfraction.den
        common = self.gcd(newnum, newden)
        return Fraction(newnum//common, newden//common)

    @staticmethod
    def gcd(num, den):
        while num%den != 0:
            oldm = num 
            oldn = den
            
            num = oldn
            den = oldm % oldn 
        return den

    def __eq__(self, other):
        firstnum = self.num * other.den 
        secondnum = other.num * self.den
        return firstnum == secondnum


f1 = Fraction(11, 7)
f1.show()
f2 = Fraction(3, 7)
f2.show()

print(f1, f2, f1+f2)
print(f1 == f2)

11 / 7
3 / 7
11/7 3/7 2/1
False
